# Neutron star

Reference: Silbar & Reddy, *Neutron Stars for Undergraduates* (https://arxiv.org/abs/nucl-th/0309041)

## Background

In this exercise, we solve for the mass-radius relation for neutron stars. We assume that the neutron star material is in hydrostatic equilibrium, i.e., assuming that the outward pressure and inward gravitational forces balance. 
In the Newtonian limit of weak gravitational fields and nonrelativistic velocities, the equation for hydrostatic equilibrium is

$$\frac{dp}{dr}= - \rho(r) \frac{G m(r)}{r^2} \qquad (1)$$

where $p(r)$ is the pressure, $\rho(r)$ is the density, $m(r)$ is the enclosed mass within a radius $r$, and $G$ is Newton's constant. Here and henceforth we work in cgs units, where $G = 6.673 \times 10^{-8} \; {\rm dyne} \; {\rm cm}^2/{\rm g}^2$ and the speed of light is $c = 3\times 10^{10} \; {\rm cm}/{\rm s}$.

We have also assumed spherical symmetry, such that all quantities depend on $r$ only.

The enclosed mass $m(r)$ can be calculated by integrating the density $\rho(r)$ over the volume of a sphere of radius $r$

$$m(r) = 4\pi \int_0^r dr^\prime r^{\prime 2} \rho(r^\prime) \; .$$

However, it is more helpful to take the derivative of this relation and relate $m(r)$ and $\rho(r)$ through a differential equation

$$\frac{dm}{dr} = 4\pi r^2 \rho(r) \; . \qquad (2)$$

In General Relativity, Eq. (1) is modified to have the following form

$$\frac{dp}{dr}= - \rho(r) \frac{G m(r)}{r^2} \left( 1 + \frac{p(r)}{\rho(r) c^2} \right)
\left( 1 + \frac{4\pi r^3 p(r)}{m(r) c^2} \right) \left( 1 - \frac{2G m(r)}{r c^2} \right)^{-1}\qquad (3)$$

known as the Tolman-Oppenheimer-Volkov equation. This equation looks very similar to Eq. (1) except for the extra correction factors being multiplied on the right-hand side.

To solve for the structure of a neutron star, we need to solve for

$$p(r), \; m(r) , \; \rho(r)$$

by solving either Eqs. (1) and (2) in the Newtonian limit, or Eqs. (1) and (3) in full General Relativity.
Since we have three unknown functions, we must also impose a third relation, known as the equation of state, which is a relationship between pressure and density.

Here, we adopt a quadratic equation of state following the reference given above (see Eq. (96) therein and converting to cgs units)

$$p(r) = K_0 \rho^2 \qquad (4)$$

where the constant $K_0 = 2.03 \times 10^5$ for $p$, $\rho$ given in cgs units. 

In solving these equations, we need to impose boundary conditions. The strategy is to fix the boundary conditions at the origin, $r=0$, and solve for $p(r)$ and $m(r)$ for $r > 0$. The boundary conditions are

$$p(0) = p_0 , \quad m(0) = 0 \, .$$

Since $dp/dr$ is negative, the pressure will decrease with $r$. Where the pressure goes to zero, that defines the radius of the neutron star $R$, i.e.,

$$p(R) = 0$$

The total mass of the neutron star $M$ is the mass enclosed within $R$, i.e.,

$$m(R) = M$$

The values of $M$-$R$ that one obtains, as a function of the initial conditions, is the *mass-radius relation*.



## Part A

Solve the system of coupled ODEs (1) and (2) corresponding to neutron stars in the Newtonian limit, taking a central density $\rho_0 = 10^{15} \; {\rm g}/{\rm cm}^3$.

Compute the total mass $M$ and radius $R$. *Note:* Values of $M$ will be in units of grams. Convert $M$ to units of solar masses. Also, $R$ will be in units of ${\rm cm}$; convert $R$ to units of ${\rm km}$.

Next, repeat the calculation above in the case of full General relativity, by solving the coupled ODEs (2) and (3), with the same initial conditions.

Make a log-log plot that shows $\rho(r)$ as a function of $r$ for the two cases (Newtonian and General Relativity).

*Hints:* 

1. You can write your own ODE solver, or you can use `solve_ivp` (see documentation [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html)). For `solve_ivp`, note that here the independent variable `t` is $r$, and the dependent variable `y` is $$y = \left(\begin{array}{c} p(r) \\ m(r) \end{array}\right)\, .$$
2. Since Eq. (1) is ill-defined exactly at $r=0$ (dividing by 0), impose your boundary condition at a small nonzero radius $r_0$. Take $$p(r_0) = p_0 = K_0 \rho_0^2 \, , \quad m(r_0) = m_0 = \frac{4\pi}{3} \rho_0 r_0^3$$
for your initial conditions, and solve for $p(r)$ and $m(r)$ for $r > r_0$.
3. You want to stop your ODE solver when the pressure drops to zero, i.e., you go from $r=r_0$ to $r=R$, where $R$ is defined by $p(R)=0$. This can be accomplished in `solve_ivp` using the `events` keyword, as follows. First, define a function `event_function(t,y)` that returns the pressure. Second, include the line of code

```py
event_function.terminal = True
```

which tells `solve_ivp` to stop the integration if the pressure changes sign. Third, include the keyword `events=event_func` in `solve_ivp`.



In [12]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

# Define constants
G = 6.673e-8 # dyne-cm^2/g^2
c = 3e10 # cm/s
M_sun = 1.989e33 # solar mass in grams
K0 = 2.03e5
rho0 = 10e15
r0 = 0.1 # initial radius in cm
# Your code here
def ode_newton(r,y): # r is test radii, y will be an array of pressure and mass newtonian method
    p = y[0] # pressure p
    m = y[1] # mass m
    rho = np.sqrt(p / K0)
    dp = - rho * G * m / r**2 #classical functions
    dm = 4 * np.pi * r**2 * rho
    return np.array([dp,dm])

def event_function(r,y):
    return y[0] # checks pressure
event_function.terminal = True
r_span = [r0, 10**12]# The radius of neutron are between 10-20 km, which is 10^6 cm.
y0 = np.array([K0 * rho0**2,(4 * np.pi / 3) * rho0 * r0**3])
event_function.terminal = True #if sign flips, terminates
sol_newton = solve_ivp(ode_newton, t_span=r_span, y0=y0, events=event_function) #r0 is very small
print('Newton:')
print(sol_newton) # first array is the radii (km), 2nd is the pressures at each radii, then 3rd is the the (total?) mass eclosed at radii, however this pressure doesnt make much sense as it never goes to 0 but this could be rounding errors

Newton:
  message: 'Required step size is less than spacing between numbers.'
     nfev: 1412
     njev: 0
      nlu: 0
      sol: None
   status: -1
  success: False
        t: array([1.00000000e-01, 1.29803568e-01, 4.27839250e-01, 3.40819607e+00,
       3.32117642e+01, 3.31247446e+02, 3.31160426e+03, 1.85122571e+04,
       6.14250485e+04, 1.51314061e+05, 3.08345187e+05, 5.50476333e+05,
       8.88030049e+05, 1.31587848e+06, 1.63642945e+06, 1.70053964e+06,
       1.76464983e+06, 1.83099726e+06, 1.89734469e+06, 1.94376447e+06,
       1.99018425e+06, 2.02177352e+06, 2.05336278e+06, 2.07495553e+06,
       2.09654829e+06, 2.11128115e+06, 2.12601401e+06, 2.13599582e+06,
       2.14597762e+06, 2.15268225e+06, 2.15938688e+06, 2.16385373e+06,
       2.16832059e+06, 2.17127652e+06, 2.17423245e+06, 2.17617843e+06,
       2.17812440e+06, 2.17940067e+06, 2.18067694e+06, 2.18151177e+06,
       2.18234660e+06, 2.18289168e+06, 2.18343677e+06, 2.18379223e+06,
       2.18414770e+06, 2.18437933e+06, 2.

/tmp/ipykernel_4904/143343093.py:16: RuntimeWarning: invalid value encountered in sqrt
  rho = np.sqrt(p / K0)


In [8]:
def ode_relativity(r,y):
    p = y[0]
    m = y[1] # indepdent variables
    rho = np.sqrt(p / K0)
    dm = 4 * np.pi * r**2 * rho
    dp = (-rho * G * m / r**2) * (1 + (p / (rho * c**2))) * (1 + ((4 * np.pi * r**3 * p) / (m * c**2))) * (1 - ((2 * G * m) / (r * c**2)))**(-1) # general relativity function

    return np.array([dp,dm])

ode_relativity.terminal = True
sol_relativity = solve_ivp(ode_relativity, t_span=r_span,y0=y0)
print('General Relativity:')
print(sol_relativity)

General Relativity:
  message: 'Required step size is less than spacing between numbers.'
     nfev: 1448
     njev: 0
      nlu: 0
      sol: None
   status: -1
  success: False
        t: array([1.00000000e-01, 1.29803568e-01, 4.27839250e-01, 3.40819607e+00,
       3.32117642e+01, 3.31247446e+02, 3.31160426e+03, 1.51995839e+04,
       4.50939624e+04, 1.02621935e+05, 1.93337638e+05, 3.13316726e+05,
       4.22645575e+05, 5.23410804e+05, 6.23105393e+05, 6.99145809e+05,
       7.63668336e+05, 7.76572842e+05, 7.89477347e+05, 8.02942918e+05,
       8.16408488e+05, 8.27616739e+05, 8.38824989e+05, 8.47745876e+05,
       8.56666763e+05, 8.63506572e+05, 8.70346382e+05, 8.75413837e+05,
       8.80481293e+05, 8.84123649e+05, 8.87766005e+05, 8.90317999e+05,
       8.92869993e+05, 8.94621644e+05, 8.96373295e+05, 8.97556729e+05,
       8.98740163e+05, 8.99530402e+05, 9.00320640e+05, 9.00843924e+05,
       9.01367208e+05, 9.01711705e+05, 9.02056201e+05, 9.02282095e+05,
       9.02507989e+05, 9.0265

/tmp/ipykernel_4904/1706922007.py:4: RuntimeWarning: invalid value encountered in sqrt
  rho = np.sqrt(p / K0)


In [13]:
plt.xlim(1, 30)
plt.xlabel('Radius (km)')
plt.ylabel('Mass (solar mass)')
plt.loglog(sol_newton.t / 1e5, np.sqrt(sol_newton.y[0] / K0), label='Newton')
plt.loglog(sol_relativity.t/1e5,np.sqrt(sol_relativity.y[0] / K0), label='General Relativity')
plt.legend()
print('Radius in Newton:')
print(np.max(sol_newton.t / 1e5), 'km') # 21km
print('Radius in General Relativity:')
print(np.max(sol_relativity.t/1e5), 'km') # 9km
print('Mass in Newton:')
print(np.max(sol_newton.y[1]) / M_sun, 'solar mass') # 66 solar masses in 21km seems too high
print('Mass in General Relativity:')
print(np.max(sol_relativity.y[1] / M_sun), 'solar mass') # 1.9 solar masses, much more reasonable
#Pressure falls off a cliff once a certain radius is reached. Lower for general relativity which makes sense as most neutrons stars are 1 or 2 solar masses.

Radius in Newton:
21.85475027206657 km
Radius in General Relativity:
9.033568414150183 km
Mass in Newton:
66.82293670018339 solar mass
Mass in General Relativity:
1.9078215523868574 solar mass


## Part B

Consider a range of central densty values defined by the following array:

```py
rho0_list = np.logspace(14,16)
```

i.e., values of $\rho_0$ between $10^{14}$. For all values of $\rho_0$ given above, compute the values of $M$ (in solar masses) and $R$ (in ${\rm km}$) for *both* cases of Newtonian and General Relativistic neutron stars. Make a plot of $M$ (on the y-axis) and $R$ (on the x-axis), labeling the two cases. Use a log-scale for the y-axis and a linear scale for the x-axis.

Determine the maximum neutron star mass $M$ according to your calculations in the case of full General Relativity.

*Hint:* You should find that the Newton radius $R$ is independent of $M$, and that there is no maximum mass. 



In [14]:
rho0_list = np.logspace(14,16)
radii_relativity = []# Will store the corresponding radius at which the pressure reaches 0, ie where mass is greatest
masses_relativity = [] # Will store the maxiumum mass of each different density which will allow for graphing
radii_newton = []
masses_newton = []
for i in range(len(rho0_list)):
    yinit = np.array([K0 * rho0_list[i]**2, (4 * np.pi / 3) * rho0_list[i] * r0**3])  # define initial conditions
    #Relativity
    sol_relativity=solve_ivp(ode_relativity, t_span=r_span,y0=yinit,events=event_function) # does IVP for each rho value
    radii_relativity.append(sol_relativity.t[-1]) # take last element in radii
    masses_relativity.append(sol_relativity.y[1][-1]) # take last element in the 2nd element of a 2d array
    #Newton
    sol_newton = solve_ivp(ode_newton, t_span=r_span, y0=yinit, events=event_function) #r0 is very small
    radii_newton.append(sol_newton.t[-1]) # take last element in radii
    masses_newton.append(sol_newton.y[1][-1]) # take last element in the 2nd element of a 2d array

radii_relativity, masses_relativity, radii_newton, masses_newton = np.array(radii_relativity), np.array(masses_relativity), np.array(radii_newton), np.array(masses_newton)
radii_relativitykm, radii_newtonkm = radii_relativity / 1e5, radii_newton / 1e5
masses_relativityM0, masses_newtonM0 = masses_relativity / M_sun, masses_newton / M_sun #Converts our data to numpy arrays, and coverts them to appropriate units

plt.xlabel('Radius (km)')
plt.ylabel('Mass (solar mass)')
plt.plot(radii_relativitykm, masses_relativityM0, label='General Relativity')
plt.plot(radii_newtonkm, masses_newtonM0, label='Newton') #Straight line, agrees with hint, there is no maxiumum.
plt.yscale('log')
plt.legend()
print("Given the different starting densities ranging from 10^14 to 10^16 g/cm^3, we found the maximum mass to be "+str(np.max(masses_relativity)/M_sun)+"M0. This occurs at a density of "+str(rho0_list[np.argmax(masses_relativity)])) #don't know how to put this in scientific notation, stack-overflow not helpful

/tmp/ipykernel_3270/1706922007.py:4: RuntimeWarning: invalid value encountered in sqrt
  rho = np.sqrt(p / K0)
/tmp/ipykernel_3270/1193190197.py:16: RuntimeWarning: invalid value encountered in sqrt
  rho = np.sqrt(p / K0)


Given the different starting densities ranging from 10^14 to 10^16 g/cm^3, we found the maximum mass to be 2.3591160451158943M0. This occurs at a density of 1842069969326716.2


| Task |     Points |                Comments |
| :--------------------------------------------------------------- | :-----------: | :-----------------------------------: |
| Step 1: Compute M\-R relation in Newtonian limit & GR |      4/4 |  |
| Step 2: Determine total NS mass and radius in appropriate units |      1/2 | Wrong radius in GR and mass in Newton |
| Step 3: Compute and plot M and R for given density values |      3/3 |  |
| Step 4: Evaluate maximum NS mass |      1/1 |  |
| Bonus: Good work practice, axis labels, meaningful comments etc. |     0.5/1 |   Wrong axis label in pressure plot |
| Total | 9\(\+0.5\)/10 |               Very good! |

